Write a wrapper class TableData for database table, that when initialized with database name and table acts as collection object (implements Collection protocol).
Assume all data has unique values in 'name' column.
So, if presidents = TableData(database_name='example.sqlite', table_name='presidents')

then
 -  `len(presidents)` will give current amount of rows in presidents table in database
 -  `presidents['Yeltsin']` should return single data row for president with name Yeltsin
 -  `'Yeltsin' in presidents` should return if president with same name exists in table
 -  object implements iteration protocol. i.e. you could use it in for loops::
       for president in presidents:
           print(president['name'])
 - all above mentioned calls should reflect most recent data. If data in table changed after you created collection instance, your calls should return updated data.

Avoid reading entire table into memory. When iterating through records, start reading the first record, then go to the next one, until records are exhausted.
When writing tests, it's not always neccessary to mock database calls completely. Use supplied example.sqlite file as database fixture file.

In [ ]:
import sqlite3

class TableData:
    def __init__(self, database_name, table_name):
        self.database_name = database_name
        self.table_name = table_name

    # подключение к базе
    def _get_connection(self):
        return sqlite3.connect(self.database_name)

    # количество строк
    def __len__(self):
        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM {self.table_name}")
            return cursor.fetchone()[0]

    # информация об элементе
    def __getitem__(self, name):
        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT * FROM {self.table_name} WHERE name=?", (name,))
            row = cursor.fetchone()
            if row is None:
                raise KeyError(f"No record found with name '{name}'")
            return dict(zip([column[0] for column in cursor.description], row))

    # проверка на сущестование элемента
    def __contains__(self, name):
        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT 1 FROM {self.table_name} WHERE name=?", (name,))
            return cursor.fetchone() is not None

    # вывод элементов в списке
    def __iter__(self):
        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute(f"SELECT * FROM {self.table_name}")
            columns = [column[0] for column in cursor.description]
            for row in cursor:
                yield dict(zip(columns, row))

# Пример использования класса
if __name__ == "__main__":
    db = 'example.sqlite'
    presidents = TableData(database_name=db, table_name='presidents')

    # Получение количества президентов
    count = len(presidents)
    print(f"Number of presidents: {count}")

    # Получение данных о президенте
    yeltsin = presidents['Yeltsin']
    print(f"Yeltsin's data: {yeltsin}")

    # Проверка наличия президента
    exists_yeltsin = 'Yeltsin' in presidents
    print(f"Is Yeltsin in presidents? {exists_yeltsin}")
    exists_putin = 'Putin' in presidents
    print(f"Is Putin in presidents? {exists_putin}")

    # Итерация по всем президентам
    print("List of presidents:")
    for president in presidents:
        print(president['name'])

Number of presidents: 4
Yeltsin's data: {'name': 'Yeltsin', 'age': 999, 'country': 'Russia'}
Is Yeltsin in presidents? True
Is Putin in presidents? False
List of presidents:
Yeltsin
Trump
Big Man Tyrone
Truman
